# Trustpilot scraping - Trade Republic

Este notebook scrapea reseñas de Trustpilot para `www.traderepublic.com` desde la pagina 3 hasta la 10, extrayendo texto y fecha, y guardando el resultado en CSV.

In [7]:
# Si falta alguna dependencia, descomenta y ejecuta:
# !pip install requests beautifulsoup4 pandas lxml

In [8]:
import time
import re
from typing import List, Dict

import requests
import pandas as pd
from bs4 import BeautifulSoup


In [9]:
BASE_URL = "https://es.trustpilot.com/review/www.traderepublic.com"
START_PAGE = 1
END_PAGE = 10
OUTPUT_CSV = "traderepublic_reviews_p1_p10.csv"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
}


In [10]:
def parse_reviews_from_html(html: str) -> List[Dict[str, str]]:
    soup = BeautifulSoup(html, "lxml")

    reviews_data = []

    # Trustpilot cambia con frecuencia el atributo del contenedor de rese?a.
    review_cards = soup.select("div[data-testid='service-review-card-v2']")
    if not review_cards:
        review_cards = soup.select("article[data-service-review-card-paper]")

    for card in review_cards:
        text_elem = card.select_one("p[data-service-review-text-typography]")
        date_elem = card.select_one("time[data-service-review-date-time-ago]") or card.select_one("time")

        # La puntuaci?n suele venir en el alt, por ejemplo: "Valorada con 1 estrellas sobre 5".
        rating_elem = card.select_one("img.CDS_StarRating_starRating__614d2e[alt]") or card.select_one("img[alt*='sobre 5']")

        review_text = text_elem.get_text(" ", strip=True) if text_elem else ""
        review_date = date_elem.get("datetime", "").strip() if date_elem else ""

        review_rating = ""
        if rating_elem:
            alt_text = rating_elem.get("alt", "")
            match = re.search(r"(\d+)\s+estrellas?\s+sobre\s+5", alt_text, flags=re.IGNORECASE)
            if match:
                review_rating = match.group(1)
            else:
                src = rating_elem.get("src", "")
                src_match = re.search(r"stars-(\d)\.svg", src)
                if src_match:
                    review_rating = src_match.group(1)

        if review_text:
            reviews_data.append({
                "review_text": review_text,
                "review_date": review_date,
                "review_rating": review_rating,
            })

    return reviews_data


In [11]:
all_reviews = []

for page in range(START_PAGE, END_PAGE + 1):
    url = f"{BASE_URL}?page={page}"
    print(f"Scrapeando: {url}")

    response = requests.get(url, headers=HEADERS, timeout=30)
    response.raise_for_status()

    page_reviews = parse_reviews_from_html(response.text)
    print(f"  -> Reseñas encontradas: {len(page_reviews)}")

    all_reviews.extend(page_reviews)
    time.sleep(1.5)

print(f"\nTotal reseñas extraidas: {len(all_reviews)}")


Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=1
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=2
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=3
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=4
  -> Reseñas encontradas: 19
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=5
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=6
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=7
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=8
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=9
  -> Reseñas encontradas: 20
Scrapeando: https://es.trustpilot.com/review/www.traderepublic.com?page=1

In [12]:
df = pd.DataFrame(all_reviews, columns=["review_text", "review_date", "review_rating"])
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"CSV guardado en: {OUTPUT_CSV}")
df.head()


CSV guardado en: traderepublic_reviews_p1_p10.csv


,review_text,review_date
0,LLevo mas de 2 año intentando contactar con el...,2026-02-20T23:19:03.000Z
1,"Ojo con esto, yo solo expongo mi experiencia. ...",2026-02-20T01:03:54.000Z
2,Las recompensas por invitar gente no funcionan...,2026-02-19T12:45:51.000Z
3,Publicidad engañosa y mal servicio de atención...,2026-02-17T13:24:56.000Z
4,Desde hace semanas no puedo realizar una trans...,2026-02-16T11:51:14.000Z
